In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from helper_functions import *

In [2]:
FILEPATH = "/Users/carlosito/Library/CloudStorage/OneDrive-Personal/Personal Document/Hexamind/hexamind_code/Carrefour/Clustering/Trustadvisor_Playground/reviews.csv"
df = pd.read_csv(FILEPATH, index_col=0)
df.head()

,titles,ratings,reviews,dates,replies,reply_dates
0,Mais ou est ma commande???,1,"Commande N° 609249735,toujours non livrée,pass...",2023-02-07T17:39:04.000Z,"Bonjour, nous vous informons que nous avons pr...",2023-02-08T09:48:06.000Z
1,Cela fait plusieurs semaines même plus…,1,Magasin de Carrefour Trans en Provence 83\nCel...,2023-02-06T14:48:53.000Z,NaN,NaN
2,Commande Deliveroo annulé !,1,Commande Deliveroo passé ce jour sans suite ca...,2023-02-08T10:54:56.000Z,"Bonjour, nous vous remercions pour cette infor...",2023-02-08T12:59:53.000Z
3,une honte cette société qu'utilise…,1,une honte cette société qu'utilise Carrefour p...,2023-02-04T18:29:49.000Z,"Bonjour, pourriez-vous nous communiquer votre ...",2023-02-05T08:17:54.000Z
4,Tout se passe bien chez Carrefour City…,1,Tout se passe bien chez Carrefour City sauf po...,2023-02-05T11:41:45.000Z,"Bonjour, pouvez-vous nous préciser quel magasi...",2023-02-04T16:18:10.000Z


In [3]:
df.shape

(2000, 6)

In [5]:
clean_df = remove_and_combined(df,
                    title_col='titles',
                    review_col='reviews')
clean_df.to_csv('clean_df.csv')